In [1]:
import os
import sys
proj_path = os.path.abspath(os.path.join('..'))
if proj_path not in sys.path:
    sys.path.append(proj_path)

# Create first network with Keras
import numpy as np
from scipy.spatial.distance import pdist, squareform
from keras.models import Sequential
from keras.layers import Dense, Merge
import numpy as np
import pandas as pd
from keras import backend as K
import tensorflow as tf
from csrank.util import *
from csrank.metrics import *
from csrank.dataset_reader import SyntheticDatasetGenerator
from scipy.stats import kendalltau, spearmanr, rankdata
from itertools import combinations
from sklearn.utils import check_random_state
rs = np.random.RandomState(seed=42)

/home/prithag/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/prithag/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


n_objects = 5
n_instances = 5
b = np.array([rs.choice(n_objects, replace=False, size=n_objects) for _ in range(n_instances)])
c = rs.uniform(size=(n_instances,n_objects))
orderings = np.array([rankdata(pred).astype(int)-1 for pred in c])
print(b)
orderings[0] = b[0]
print(orderings)
print(np.equal(orderings,b))
print(rankdata([0, 5, 3.5, 2.5])-1)
np.all(np.equal(orderings,b),axis=1)


In [2]:
with tf.Session() as sess:
    n_objects = 3
    n_instances = 10
    n = K.placeholder(dtype='int32', name='n')
    b = K.placeholder(dtype='float32', shape=(None, n_objects), name='b')
    c = K.placeholder(dtype='float32', shape=(None, n_objects), name='c')
    result = kendalls_tau_for_scores(b, c)
    result2 = spearman_correlation_for_scores(b, c)
    result3 = zero_one_rank_loss_for_scores(b,c)
    result4 = get_rankings_tensor(n,c)
    b_fill = np.array([rs.choice(n_objects, replace=False, size=n_objects) for _ in range(n_instances)])
    c_fill = rs.uniform(size=(n_instances, n_objects))
    print("----------------------------------------------------------")
    print('kendall scipy:',kendalls_mean_np(b_fill,c_fill))
    print('kendall:',result.eval(feed_dict={b: b_fill, c: c_fill}))
    print("----------------------------------------------------------")
    print('spearman scipy:',spearman_mean_np(b_fill,c_fill))
    print('spearman:',result2.eval(feed_dict={b: b_fill, c: c_fill}))
    print("----------------------------------------------------------")
    print('cal rank loss:',zero_one_rank_loss_for_scores_ties_np(b_fill,c_fill))
    print('zero_one_for_scores: ',result3.eval(feed_dict={b: b_fill, c: c_fill}))

[[9.38552709e-01 9.38552709e-01 7.78765841e-04]
 [6.17481510e-01 6.17481510e-01 6.11653160e-01]
 [2.30624250e-02 2.30624250e-02 5.24774660e-01]
 [4.66656632e-02 4.66656632e-02 9.73755519e-01]
 [9.06064345e-02 9.06064345e-02 6.18386009e-01]
 [3.82461991e-01 9.83230886e-01 4.66762893e-01]
 [8.59940407e-01 6.80307539e-01 4.50499252e-01]
 [1.32649612e-02 9.42201756e-01 5.63288218e-01]
 [3.85416503e-01 1.59662522e-02 2.30893826e-01]
 [2.41025466e-01 6.83263519e-01 6.09996658e-01]]
[[1. 1. 0.]
 [1. 1. 0.]
 [0. 0. 2.]
 [0. 0. 2.]
 [0. 0. 2.]
 [0. 2. 1.]
 [2. 1. 0.]
 [0. 2. 1.]
 [2. 0. 1.]
 [0. 2. 1.]]
----------------------------------------------------------
kendall scipy: 0.09999999999999998
kendall: 0.10000008
----------------------------------------------------------
spearman scipy: -0.1633974596215561
spearman: 0.3
----------------------------------------------------------
cal rank loss: 0.45
zero_one_for_scores:  0.44999996


In [1]:
def generate_pairwise_instances(features):
    pairs = np.array(list(combinations(features, 2)))
   
    n_pairs = len(pairs)
    neg_indices = np.arange(0, n_pairs, 2)
    
    a,b = np.copy(pairs[neg_indices,0]), np.copy(pairs[neg_indices,1])
    pairs[neg_indices,1] = a
    pairs[neg_indices,0] = b
   
    X1 = pairs[:,0]
    X2 = pairs[:,1]
    Y_double = np.ones([n_pairs,1]) * np.array([1,0])
    Y_single = np.repeat(1, n_pairs)
    
    Y_double[neg_indices] = [0,1]
    Y_single[neg_indices] = 0
    return X1, X2, Y_double, Y_single

In [2]:
from csrank.dataset_reader.objectranking.util import generate_pairwise_instances
def generate_complete_pairwise_dataset(X, orderings):
    n_instances, n_objects, n_features = X.shape
    orderings = orderings.astype(int)
    X_sorted = [X[i, orderings[i], :] for i in range(n_instances)]
    Y_double = np.empty((0, 2))
    X1 = np.empty((0, n_features))
    X2 = np.empty((0, n_features))
    Y_single = np.empty((0, 1))
    for features in X_sorted:
        x1, x2, y1, y2 = generate_pairwise_instances(features)
        X1 = np.append(X1, x1, axis=0)
        X2 = np.append(X2, x2, axis=0)
        Y_double = np.append(Y_double, y1, axis=0)
        Y_single = np.append(Y_single, y2)
    X_train = X1 - X2
    return X_train, X1, X2, Y_double, Y_single

/home/prithag/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/prithag/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [19]:
import timeit
dr = SyntheticDatasetGenerator(n_train_instances=10, n_test_instances=10, n_objects=173,n_features=2)

In [20]:
X, orderings,_,_ = dr.get_single_train_test_split()
X_train, X1, X2, Y_double, Y_single = generate_complete_pairwise_dataset(X,orderings)
print(X_train.shape)
%timeit generate_complete_pairwise_dataset(X,orderings)

(148780, 2)
138 ms ± 3.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
def generate_pairwise_instances(features):
    pairs = combinations(features, 2)
    Y_double = []
    X1 = []
    X2 = []
    Y_single = []
    k = 1
    for p in pairs:
        a = np.array(p[0])
        b = np.array(p[1])
        if k == 1:
            X1.append(a)
            X2.append(b)
            Y_double.append([1, 0])
            Y_single.append(1)
        else:
            X1.append(b)
            X2.append(a)
            Y_double.append([0, 1])
            Y_single.append(0)
        k = k * -1
    X1 = np.array(X1)
    X2 = np.array(X2)
    Y_double = np.array(Y_double)
    Y_single = np.array(Y_single)
    # X1 = X1[np.newaxis, :, :]
    # X2 = X2[np.newaxis, :, :]
    return X1, X2, Y_double, Y_single

In [22]:
def generate_complete_pairwise_dataset(X, orderings):
    n_instances, n_objects, n_features = X.shape
    orderings = orderings.astype(int)
    X_sorted = [X[i, orderings[i], :] for i in range(n_instances)]
    Y_double = np.empty((0, 2))
    X1 = np.empty((0, n_features))
    X2 = np.empty((0, n_features))
    Y_single = np.empty((0, 1))
    for features in X_sorted:
        x1, x2, y1, y2 = generate_pairwise_instances(features)
        X1 = np.append(X1, x1, axis=0)
        X2 = np.append(X2, x2, axis=0)
        Y_double = np.append(Y_double, y1, axis=0)
        Y_single = np.append(Y_single, y2)
    X_train = X1 - X2
    return X_train, X1, X2, Y_double, Y_single

In [ ]:
import timeit

X, orderings,_,_ = dr.get_single_train_test_split()
%timeit generate_complete_pairwise_dataset(X,orderings)